In [248]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import json
import requests
import time
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기

---
20200325
API 활용하기

In [164]:
# 서울 각 지역별 API url과 인증키 데이터가 담긴 csv파일 읽어오기
# DataFrame(이하 DF) 형태로 생성
df_url = pd.read_csv('./seoul_salon_api_key.csv', encoding='cp949')
df_url

,url,key,district
0,http://openAPI.gangnam.go.kr:8088/(인증키)/json/GnPublicHygieneBizCosmeNomal/1/1/,6d7746646664646f313230436c6b7659,강남
1,http://openAPI.gd.go.kr:8088/(인증키)/json/GdPublicHygieneBizCosmeNomal/1/1/,744478444564646f31313163734a674d,강동
2,http://openAPI.gangbuk.go.kr:8088/(인증키)/json/GbPublicHygieneBizCosmeNomal/1/1/,414870515564646f38335946756a78,강북
3,http://openAPI.gangseo.seoul.kr:8088/(인증키)/json/GangseoPublicHygieneBizCosmeNomal/1/1/,586753714764646f3930534369636b,강서
4,http://openAPI.gwanak.go.kr:8088/(인증키)/json/GaPublicHygieneBizCosmeNomal/1/1/,676b6f4a7964646f353052464f4165,관악
5,http://openAPI.gwangjin.go.kr:8088/(인증키)/json/GwangjinPublicHygieneBizCosmeNomal/1/1/,67534f594e64646f3537546f737a62,광진
6,http://openAPI.guro.go.kr:8088/(인증키)/json/GuroPublicHygieneBizCosmeNomal/1/1/,575a6c526764646f35394a6d526e45,구로
7,http://openAPI.geumcheon.go.kr:8088/(인증키)/json/GeumcheonPublicHygieneBizCosmeNomal/1/1/,535a4b4b6564646f39316a64576573,금천
8,http://openAPI.nowon.go.kr:8088/(인증키)/json/NwPublicHygieneBizCosmeNomal/1/1/,585967667664646f37366664437a4c,노원
9,http://openAPI.dobong.go.kr:8088/(인증키)/json/DobongPublicHygieneBizCosmeNomal/1/1/,786942525164646f31367251517262,도봉


In [131]:
# DF의 value(값)만 보기 
df_url.values

array([['http://openAPI.gangnam.go.kr:8088/(인증키)/json/GnPublicHygieneBizCosmeNomal/1/1/',
        '6d7746646664646f313230436c6b7659', '강남'],
       ['http://openAPI.gd.go.kr:8088/(인증키)/json/GdPublicHygieneBizCosmeNomal/1/1/',
        '744478444564646f31313163734a674d', '강동'],
       ['http://openAPI.gangbuk.go.kr:8088/(인증키)/json/GbPublicHygieneBizCosmeNomal/1/1/',
        '414870515564646f38335946756a78', '강북'],
       ['http://openAPI.gangseo.seoul.kr:8088/(인증키)/json/GangseoPublicHygieneBizCosmeNomal/1/1/',
        '586753714764646f3930534369636b', '강서'],
       ['http://openAPI.gwanak.go.kr:8088/(인증키)/json/GaPublicHygieneBizCosmeNomal/1/1/',
        '676b6f4a7964646f353052464f4165', '관악'],
       ['http://openAPI.gwangjin.go.kr:8088/(인증키)/json/GwangjinPublicHygieneBizCosmeNomal/1/1/',
        '67534f594e64646f3537546f737a62', '광진'],
       ['http://openAPI.guro.go.kr:8088/(인증키)/json/GuroPublicHygieneBizCosmeNomal/1/1/',
        '575a6c526764646f35394a6d526e45', '구로'],
       ['http:/

In [132]:
# 'url' 컬럼에서 '(인증키)' 문자열을 'key' 컬럼값으로 치환하는 함수 생성
def modify_url(dataframe):
    return list(map(lambda x: re.sub('\(인증키\)', x[1] ,x[0]), dataframe.values))

In [133]:
# 치환함수 modify_url에 DF를 넣어 key값이 제대로 들어간 url을 list에 저장
# url 끝부분 두 개의 정수값은 요청하는 시작 데이터 번호 / 종료 데이터 번호

# 아직 총 데이터가 몇개인지 모르기 때문에
# 한개의 데이터만 불러와 총 데이터 수를 확인할 것이다.

url_list = modify_url(df_url)
url_list

['http://openAPI.gangnam.go.kr:8088/6d7746646664646f313230436c6b7659/json/GnPublicHygieneBizCosmeNomal/1/1/',
 'http://openAPI.gd.go.kr:8088/744478444564646f31313163734a674d/json/GdPublicHygieneBizCosmeNomal/1/1/',
 'http://openAPI.gangbuk.go.kr:8088/414870515564646f38335946756a78/json/GbPublicHygieneBizCosmeNomal/1/1/',
 'http://openAPI.gangseo.seoul.kr:8088/586753714764646f3930534369636b/json/GangseoPublicHygieneBizCosmeNomal/1/1/',
 'http://openAPI.gwanak.go.kr:8088/676b6f4a7964646f353052464f4165/json/GaPublicHygieneBizCosmeNomal/1/1/',
 'http://openAPI.gwangjin.go.kr:8088/67534f594e64646f3537546f737a62/json/GwangjinPublicHygieneBizCosmeNomal/1/1/',
 'http://openAPI.guro.go.kr:8088/575a6c526764646f35394a6d526e45/json/GuroPublicHygieneBizCosmeNomal/1/1/',
 'http://openAPI.geumcheon.go.kr:8088/535a4b4b6564646f39316a64576573/json/GeumcheonPublicHygieneBizCosmeNomal/1/1/',
 'http://openAPI.nowon.go.kr:8088/585967667664646f37366664437a4c/json/NwPublicHygieneBizCosmeNomal/1/1/',
 'http://

In [134]:
# 총 데이터 수 구하기 예시
# url_list의 0번째 인덱스에 저장된 강남구 url
url_list[0]

'http://openAPI.gangnam.go.kr:8088/6d7746646664646f313230436c6b7659/json/GnPublicHygieneBizCosmeNomal/1/1/'

In [135]:
# requests 모듈의 get()함수롤 사용하면 json 형태의 데이터를 가져올 수있다.
# 여기서 "list_total_count" 에 대응하는 값이 전체 데이터 수
# 이 값들을 뽑아 요청해야 할 종료 데이터 번호로 지정해줄 것이다.

requests.get(url_list[0]).text

'{"GnPublicHygieneBizCosmeNomal":{"list_total_count":1785,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"CGG_CODE":"3220000","SNT_COB_CODE":"211","YY":"1983","UPSO_SNO":"00001","SNT_COB_NM":"미용업(일반)","UPSO_GSL_YMD":"19830627","UPSO_NM":"백민재헤어샵","TRDP_AREA":19.8,"UPSO_SITE_TELNO":"02 5451290","BMAN_STDT":"20110502","BUP_NM":"","SITE_STDT":"20180515","ADMDNG_NM":"청담동","DCB_YMD":"","DCB_WHY":"","SNT_UPTAE_NM":"일반미용업","ED_FIN_YMD":"20110622","GAEKSIL":0.0,"HANSHIL":0.0,"YANGSIL":0.0,"CHAIR_NUM":3.0,"YOKSIL":0.0,"BALHANSIL_YN":"N","WASHMC_NUM":0.0,"PERM_NT_NO":"3220000-211-1983-00001","KOR_FRGNR_GBN":"내국인","NTN":"","SITE_ADDR_RD":"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)","SITE_ADDR":"서울특별시 강남구 청담동  133번지 3호  화천회관빌딩"}]}}'

In [136]:
# url_list의 url을 하나씩 requests.get()하여 json 형태로 반환
# json_list 객체에 저장
json_list = list(map(lambda x: requests.get(x).text, url_list))

In [137]:
# json_list 확인
json_list[:2]

['{"GnPublicHygieneBizCosmeNomal":{"list_total_count":1785,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"CGG_CODE":"3220000","SNT_COB_CODE":"211","YY":"1983","UPSO_SNO":"00001","SNT_COB_NM":"미용업(일반)","UPSO_GSL_YMD":"19830627","UPSO_NM":"백민재헤어샵","TRDP_AREA":19.8,"UPSO_SITE_TELNO":"02 5451290","BMAN_STDT":"20110502","BUP_NM":"","SITE_STDT":"20180515","ADMDNG_NM":"청담동","DCB_YMD":"","DCB_WHY":"","SNT_UPTAE_NM":"일반미용업","ED_FIN_YMD":"20110622","GAEKSIL":0.0,"HANSHIL":0.0,"YANGSIL":0.0,"CHAIR_NUM":3.0,"YOKSIL":0.0,"BALHANSIL_YN":"N","WASHMC_NUM":0.0,"PERM_NT_NO":"3220000-211-1983-00001","KOR_FRGNR_GBN":"내국인","NTN":"","SITE_ADDR_RD":"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)","SITE_ADDR":"서울특별시 강남구 청담동  133번지 3호  화천회관빌딩"}]}}',
 '{"GdPublicHygieneBizCosmeNomal":{"list_total_count":927,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"CGG_CODE":"3240000","SNT_COB_CODE":"211","YY":"1979","UPSO_SNO":"00001","SNT_COB_NM":"미용업(일반)","UPSO_GSL_YMD":"19791113","UPSO_NM":"꼬꼬

In [138]:
# 25개 지역별 데이터가 잘 들어왔다.
len(json_list)

25

In [139]:
# json 파일을 json 모듈의 load()함수를 통해 dict 객체로 변환
# dict_list에 저장
dict_list = list(map(lambda x: json.loads(x), json_list))
dict_list[:2]

[{'GnPublicHygieneBizCosmeNomal': {'list_total_count': 1785,
   'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다'},
   'row': [{'CGG_CODE': '3220000',
     'SNT_COB_CODE': '211',
     'YY': '1983',
     'UPSO_SNO': '00001',
     'SNT_COB_NM': '미용업(일반)',
     'UPSO_GSL_YMD': '19830627',
     'UPSO_NM': '백민재헤어샵',
     'TRDP_AREA': 19.8,
     'UPSO_SITE_TELNO': '02 5451290',
     'BMAN_STDT': '20110502',
     'BUP_NM': '',
     'SITE_STDT': '20180515',
     'ADMDNG_NM': '청담동',
     'DCB_YMD': '',
     'DCB_WHY': '',
     'SNT_UPTAE_NM': '일반미용업',
     'ED_FIN_YMD': '20110622',
     'GAEKSIL': 0.0,
     'HANSHIL': 0.0,
     'YANGSIL': 0.0,
     'CHAIR_NUM': 3.0,
     'YOKSIL': 0.0,
     'BALHANSIL_YN': 'N',
     'WASHMC_NUM': 0.0,
     'PERM_NT_NO': '3220000-211-1983-00001',
     'KOR_FRGNR_GBN': '내국인',
     'NTN': '',
     'SITE_ADDR_RD': '서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)',
     'SITE_ADDR': '서울특별시 강남구 청담동  133번지 3호  화천회관빌딩'}]}},
 {'GdPublicHygieneBizCosmeNomal': {'list_to

In [140]:
# dict 객체에서 'list_total_count'에 대응하는 총 데이터 수만 추출
# total_count_list에 저장
# 단 여기서 int 객체를 string 객체로 변환하여 저장한다.

total_count_list = list(map(lambda x: str(list(x.values())[0]['list_total_count']), dict_list))
total_count_list

['1785',
 '927',
 '1254',
 '1835',
 '1514',
 '822',
 '1309',
 '346',
 '818',
 '1073',
 '1026',
 '1206',
 '1433',
 '1031',
 '927',
 '765',
 '1355',
 '2051',
 '1324',
 '1198',
 '685',
 '1750',
 '262',
 '513',
 '938']

In [141]:
# 요청할 데이터 시작번호, 끝번호를 지정할 함수 정의
# 1회 요청 최대 데이터 건수는 1,000건이므로
# 1,000건이 넘을 시 분할 해주어야함
# ('시작번호', '끝번호') tuple로 반환

def page_range(c_number):
    val='',
    if len(c_number) < 4:
        val = ('1', c_number),
    else:
        if c_number[0] == '1':
            val = (('1', '1000'), ('1001', c_number))
        elif c_number[0] == '2':
            val =  (('1', '1000'), ('1001', '2000'), ('2001', c_number))
        elif c_number[0] == '3':
            val = (('1', '1000'), ('1001', '2000'), ('2001', '3000'), ('3001', c_number))
    return val

In [142]:
# 총 데이터 수를 정의한 page_range 함수에 넣어 요청할 데이터 시작, 끝 번호 추출
# page_list에 저장

page_list = list(map(lambda x: page_range(x), total_count_list))
page_list

[(('1', '1000'), ('1001', '1785')),
 (('1', '927'),),
 (('1', '1000'), ('1001', '1254')),
 (('1', '1000'), ('1001', '1835')),
 (('1', '1000'), ('1001', '1514')),
 (('1', '822'),),
 (('1', '1000'), ('1001', '1309')),
 (('1', '346'),),
 (('1', '818'),),
 (('1', '1000'), ('1001', '1073')),
 (('1', '1000'), ('1001', '1026')),
 (('1', '1000'), ('1001', '1206')),
 (('1', '1000'), ('1001', '1433')),
 (('1', '1000'), ('1001', '1031')),
 (('1', '927'),),
 (('1', '765'),),
 (('1', '1000'), ('1001', '1355')),
 (('1', '1000'), ('1001', '2000'), ('2001', '2051')),
 (('1', '1000'), ('1001', '1324')),
 (('1', '1000'), ('1001', '1198')),
 (('1', '685'),),
 (('1', '1000'), ('1001', '1750')),
 (('1', '262'),),
 (('1', '513'),),
 (('1', '938'),)]

In [143]:
# url에서 시작번호와 끝번호에 값을 대입하기 위해
# re.split()함수로 문자열을 잘라준다.
# split_url 에 저장

split_url = list(map(lambda x: re.split('/',x), url_list))
split_url

[['http:',
  '',
  'openAPI.gangnam.go.kr:8088',
  '6d7746646664646f313230436c6b7659',
  'json',
  'GnPublicHygieneBizCosmeNomal',
  '1',
  '1',
  ''],
 ['http:',
  '',
  'openAPI.gd.go.kr:8088',
  '744478444564646f31313163734a674d',
  'json',
  'GdPublicHygieneBizCosmeNomal',
  '1',
  '1',
  ''],
 ['http:',
  '',
  'openAPI.gangbuk.go.kr:8088',
  '414870515564646f38335946756a78',
  'json',
  'GbPublicHygieneBizCosmeNomal',
  '1',
  '1',
  ''],
 ['http:',
  '',
  'openAPI.gangseo.seoul.kr:8088',
  '586753714764646f3930534369636b',
  'json',
  'GangseoPublicHygieneBizCosmeNomal',
  '1',
  '1',
  ''],
 ['http:',
  '',
  'openAPI.gwanak.go.kr:8088',
  '676b6f4a7964646f353052464f4165',
  'json',
  'GaPublicHygieneBizCosmeNomal',
  '1',
  '1',
  ''],
 ['http:',
  '',
  'openAPI.gwangjin.go.kr:8088',
  '67534f594e64646f3537546f737a62',
  'json',
  'GwangjinPublicHygieneBizCosmeNomal',
  '1',
  '1',
  ''],
 ['http:',
  '',
  'openAPI.guro.go.kr:8088',
  '575a6c526764646f35394a6d526e45',
  'js

In [180]:
# 잘라진 split_url과 페이지 시작.끝번호 page_list를 zip()함수로 묶어준다.
combined_list = list(zip(split_url, page_list))
combined_list[:4]

[(['http:',
   '',
   'openAPI.gangnam.go.kr:8088',
   '6d7746646664646f313230436c6b7659',
   'json',
   'GnPublicHygieneBizCosmeNomal',
   '1',
   '1000',
   ''],
  (('1', '1000'), ('1001', '1785'))),
 (['http:',
   '',
   'openAPI.gd.go.kr:8088',
   '744478444564646f31313163734a674d',
   'json',
   'GdPublicHygieneBizCosmeNomal',
   '1',
   '1',
   ''],
  (('1', '927'),)),
 (['http:',
   '',
   'openAPI.gangbuk.go.kr:8088',
   '414870515564646f38335946756a78',
   'json',
   'GbPublicHygieneBizCosmeNomal',
   '1',
   '1',
   ''],
  (('1', '1000'), ('1001', '1254'))),
 (['http:',
   '',
   'openAPI.gangseo.seoul.kr:8088',
   '586753714764646f3930534369636b',
   'json',
   'GangseoPublicHygieneBizCosmeNomal',
   '1',
   '1',
   ''],
  (('1', '1000'), ('1001', '1835')))]

In [204]:
district_list = list(df_url['district'].values)
district_list

['강남',
 '강동',
 '강북',
 '강서',
 '관악',
 '광진',
 '구로',
 '금천',
 '노원',
 '도봉',
 '동대문',
 '동작',
 '마포',
 '서대문',
 '서초',
 '성동',
 '성북',
 '송파',
 '양천',
 '영등포',
 '용산',
 '은평',
 '종로',
 '중구',
 '중랑']

In [184]:
# 시작번호와 끝번호에 각각의 값을 대입
# 완성된 최종 url을 final_url에 저장한다.
final_url=[]
for i, v in enumerate(combined_list):
    for page in v[1]:
        v[0][-3] = page[0]
        v[0][-2] = page[1]
        final_url.append(('/'.join(v[0]), district_list[i]))

In [185]:
# final_url 확인
final_url[:3]

[('http://openAPI.gangnam.go.kr:8088/6d7746646664646f313230436c6b7659/json/GnPublicHygieneBizCosmeNomal/1/1000/',
  '강남'),
 ('http://openAPI.gangnam.go.kr:8088/6d7746646664646f313230436c6b7659/json/GnPublicHygieneBizCosmeNomal/1001/1785/',
  '강남'),
 ('http://openAPI.gd.go.kr:8088/744478444564646f31313163734a674d/json/GdPublicHygieneBizCosmeNomal/1/927/',
  '강동')]

In [186]:
# 최종 url 개수 확인
# 요청할 회수 / requests.get()함수로 데이터를 읽을 회수와 같다.
len(final_url)

41

---
### 완성된 url로 이제 진짜 전체 데이터를 조회해보자!

In [187]:
# 최종 완성된 url로 다시 json파일을 요청
json_list = list(map(lambda x: (x[1], requests.get(x[0]).text), final_url))

In [188]:
# 요청하여 받은 json 파일을 dict 객체로 변환
dict_list = list(map(lambda x: (x[0], json.loads(x[1])), json_list))

In [189]:
# 에러 없이 데이터 조회가 되었는지 확인 - 'RESULT' key로 확인
error_dict = list(map(lambda x: list(x[1].values())[0]['RESULT'], dict_list))
DataFrame(error_dict)

,CODE,MESSAGE
0,INFO-000,정상 처리되었습니다
1,INFO-000,정상 처리되었습니다
2,INFO-000,정상 처리되었습니다
3,INFO-000,정상 처리되었습니다
4,INFO-000,정상 처리되었습니다
5,INFO-000,정상 처리되었습니다
6,INFO-000,정상 처리되었습니다
7,INFO-000,정상 처리되었습니다
8,INFO-000,정상 처리되었습니다
9,INFO-000,정상 처리되었습니다


In [171]:
# 41번 모두 에러없이 조회되었다.
# 에러 발생시 다음 정보를 확인할 것
pd.read_csv('./seoul_salon_error_code.csv', encoding='cp949')

,code,state
0,INFO-000,정상 처리되었습니다
1,ERROR-300,필수 값이 누락되어 있습니다. 요청인자를 참고 하십시오.
2,INFO-100,"인증키가 유효하지 않습니다. 인증키가 없는 경우, 열린 데이터 광장 홈페이지에서 인증키를 신청하십시오."
3,ERROR-301,파일타입 값이 누락 혹은 유효하지 않습니다. 요청인자 중 TYPE을 확인하십시오.
4,ERROR-310,해당하는 서비스를 찾을 수 없습니다. 요청인자 중 SERVICE를 확인하십시오.
5,ERROR-331,요청시작위치 값을 확인하십시오. 요청인자 중 START_INDEX를 확인하십시오.
6,ERROR-332,요청종료위치 값을 확인하십시오. 요청인자 중 END_INDEX를 확인하십시오.
7,ERROR-333,요청위치 값의 타입이 유효하지 않습니다. 요청위치 값은 정수를 입력하세요.
8,ERROR-334,요청종료위치 보다 요청시작위치가 더 큽니다. 요청시작조회건수는 정수를 입력하세요.
9,ERROR-335,샘플데이터(샘플키:sample) 는 한번에 최대 5건을 넘을 수 없습니다. 요청시작위치와 요청종료위치 값은 1 ~ 5 사이만 가능합니다.


In [197]:
# 필요한 데이터는 'row' key에 대응하는 value이다.
# 조회 예시
list(dict_list[0][1].values())[0]['row'][:2]

[{'CGG_CODE': '3220000',
  'SNT_COB_CODE': '211',
  'YY': '1983',
  'UPSO_SNO': '00001',
  'SNT_COB_NM': '미용업(일반)',
  'UPSO_GSL_YMD': '19830627',
  'UPSO_NM': '백민재헤어샵',
  'TRDP_AREA': 19.8,
  'UPSO_SITE_TELNO': '02 5451290',
  'BMAN_STDT': '20110502',
  'BUP_NM': '',
  'SITE_STDT': '20180515',
  'ADMDNG_NM': '청담동',
  'DCB_YMD': '',
  'DCB_WHY': '',
  'SNT_UPTAE_NM': '일반미용업',
  'ED_FIN_YMD': '20110622',
  'GAEKSIL': 0.0,
  'HANSHIL': 0.0,
  'YANGSIL': 0.0,
  'CHAIR_NUM': 3.0,
  'YOKSIL': 0.0,
  'BALHANSIL_YN': 'N',
  'WASHMC_NUM': 0.0,
  'PERM_NT_NO': '3220000-211-1983-00001',
  'KOR_FRGNR_GBN': '내국인',
  'NTN': '',
  'SITE_ADDR_RD': '서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)',
  'SITE_ADDR': '서울특별시 강남구 청담동  133번지 3호  화천회관빌딩'},
 {'CGG_CODE': '3220000',
  'SNT_COB_CODE': '211',
  'YY': '1983',
  'UPSO_SNO': '00001',
  'SNT_COB_NM': '미용업(일반)',
  'UPSO_GSL_YMD': '19830627',
  'UPSO_NM': '백민재헤어샵',
  'TRDP_AREA': 65.67,
  'UPSO_SITE_TELNO': '02 5451290',
  'BMAN_STDT': '20110502',
  'BUP_N

In [198]:
# 조회한 데이터에서 필요한 'row' key에 해당하는 값만 뽑아오기
real_dict = list(map(lambda x: list(x[1].values())[0]['row'], dict_list))

In [205]:
district_list = list(map(lambda x: x[0], dict_list))
district_list

['강남',
 '강남',
 '강동',
 '강북',
 '강북',
 '강서',
 '강서',
 '관악',
 '관악',
 '광진',
 '구로',
 '구로',
 '금천',
 '노원',
 '도봉',
 '도봉',
 '동대문',
 '동대문',
 '동작',
 '동작',
 '마포',
 '마포',
 '서대문',
 '서대문',
 '서초',
 '성동',
 '성북',
 '성북',
 '송파',
 '송파',
 '송파',
 '양천',
 '양천',
 '영등포',
 '영등포',
 '용산',
 '은평',
 '은평',
 '종로',
 '중구',
 '중랑']

In [153]:
# dict 객체를 DF로 변환
# df_list에 저장
df_list = list(map(lambda x: DataFrame(x), real_dict))

In [154]:
# df_list 조회 - 0번째 DF는 '강남구' 미용실 정보
df_list[0].head(2)

,CGG_CODE,SNT_COB_CODE,YY,UPSO_SNO,SNT_COB_NM,UPSO_GSL_YMD,UPSO_NM,TRDP_AREA,UPSO_SITE_TELNO,BMAN_STDT,...,YANGSIL,CHAIR_NUM,YOKSIL,BALHANSIL_YN,WASHMC_NUM,PERM_NT_NO,KOR_FRGNR_GBN,NTN,SITE_ADDR_RD,SITE_ADDR
0,3220000,211,1983,00001,미용업(일반),19830627,백민재헤어샵,19.80,02 5451290,20110502,...,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩
1,3220000,211,1983,00001,미용업(일반),19830627,백민재헤어샵,65.67,02 5451290,20110502,...,0.0,3.0,0.0,N,0.0,3220000-211-1983-00001,내국인,,"서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)",서울특별시 강남구 청담동 133번지 3호 화천회관빌딩


In [206]:
# df_list에 저장된 모든 DF에 '지역' 상위 인덱스 추가하기
for i, df in enumerate(df_list):
    df_list[i].index = [[district_list[i]]*len(df_list[i]), df_list[i].index]

In [207]:
# 계층형 인덱스의 상위 인덱스 지역명이 잘 들어간 모습
df_list[0].head(2)

CGG_CODE SNT_COB_CODE    YY UPSO_SNO SNT_COB_NM UPSO_GSL_YMD UPSO_NM  \
강남 0  3220000  211          1983  00001    미용업(일반)    19830627     백민재헤어샵   
   1  3220000  211          1983  00001    미용업(일반)    19830627     백민재헤어샵   

      TRDP_AREA UPSO_SITE_TELNO BMAN_STDT  ... YANGSIL CHAIR_NUM YOKSIL  \
강남 0  19.80      02 5451290      20110502  ...  0.0     3.0       0.0     
   1  65.67      02 5451290      20110502  ...  0.0     3.0       0.0     

     BALHANSIL_YN WASHMC_NUM              PERM_NT_NO KOR_FRGNR_GBN  NTN  \
강남 0  N            0.0        3220000-211-1983-00001  내국인                 
   1  N            0.0        3220000-211-1983-00001  내국인                 

                                      SITE_ADDR_RD  \
강남 0  서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   
   1  서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   

                            SITE_ADDR  
강남 0  서울특별시 강남구 청담동  133번지 3호  화천회관빌딩  
   1  서울특별시 강남구 청담동  133번지 3호  화천회관빌딩  

[2 rows x 29 columns]

In [209]:
# DF 하나로 병합
df_all = pd.concat(df_list)

In [225]:
# 병합된 DF 확인
df_all.head(3)

CGG_CODE SNT_COB_CODE    YY UPSO_SNO SNT_COB_NM UPSO_GSL_YMD UPSO_NM  \
강남 0  3220000  211          1983  00001    미용업(일반)    19830627     백민재헤어샵   
   1  3220000  211          1983  00001    미용업(일반)    19830627     백민재헤어샵   
   2  3220000  211          1983  00001    미용업(일반)    19830627     백민재헤어샵   

      TRDP_AREA UPSO_SITE_TELNO BMAN_STDT  ... YANGSIL CHAIR_NUM YOKSIL  \
강남 0  19.80      02 5451290      20110502  ...  0.0     3.0       0.0     
   1  65.67      02 5451290      20110502  ...  0.0     3.0       0.0     
   2  19.80      02 5451290      20110502  ...  0.0     3.0       0.0     

     BALHANSIL_YN WASHMC_NUM              PERM_NT_NO KOR_FRGNR_GBN  NTN  \
강남 0  N            0.0        3220000-211-1983-00001  내국인                 
   1  N            0.0        3220000-211-1983-00001  내국인                 
   2  N            0.0        3220000-211-1983-00001  내국인                 

                                      SITE_ADDR_RD  \
강남 0  서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   
   1  서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   
   2  서울특별시 강남구 삼성로 649, (삼성동,(상아APT 상가 107호))       

                                    SITE_ADDR  
강남 0  서울특별시 강남구 청담동  133번지 3호  화천회관빌딩          
   1  서울특별시 강남구 청담동  133번지 3호  화천회관빌딩          
   2  서울특별시 강남구 삼성동  19번지 4호  (상아APT 상가 107호)  

[3 rows x 29 columns]

In [221]:
# 컬럼을 보기 쉽게 변환하기
# 컬럼명 정보가 담긴 csv 파일 읽어오기 
col_convert = pd.read_csv('./seoul_salon_column_info.csv', encoding='cp949')
col_convert

,출력명,출력설명
0,CGG_CODE,시군구코드
1,SNT_COB_CODE,업종코드
2,YY,년도
3,UPSO_SNO,업소일련번호
4,SNT_COB_NM,업종명
5,UPSO_GSL_YMD,신고일자
6,UPSO_NM,업소명
7,TRDP_AREA,면적
8,UPSO_SITE_TELNO,소재지전화번호
9,BMAN_STDT,영업자시작일


In [230]:
# 두 컬럼의 값을 dict 객체로 변환
col_dict = dict(zip(col_convert.출력명, col_convert.출력설명))
col_dict

{'CGG_CODE': '시군구코드',
 'SNT_COB_CODE': '업종코드',
 'YY': '년도',
 'UPSO_SNO': '업소일련번호',
 'SNT_COB_NM': '업종명',
 'UPSO_GSL_YMD': '신고일자',
 'UPSO_NM': '업소명',
 'TRDP_AREA': '면적',
 'UPSO_SITE_TELNO': '소재지전화번호',
 'BMAN_STDT': '영업자시작일',
 'BUP_NM': '법인명',
 'SITE_STDT': '소재지시작일',
 'ADMDNG_NM': '행정동명',
 'DCB_YMD': '폐업일자',
 'DCB_WHY': '폐업사유',
 'SNT_UPTAE_NM': '업태명',
 'ED_FIN_YMD': '위생교육수료일자',
 'GAEKSIL': '객실수',
 'HANSHIL': '한실수',
 'YANGSIL': '양실수',
 'CHAIR_NUM': '의자수',
 'YOKSIL': '욕실수',
 'BALHANSIL_YN': '발한실',
 'WASHMC_NUM': '세탁기수',
 'PERM_NT_NO': '허가(신고)번호',
 'KOR_FRGNR_GBN': '내외국인구분',
 'NTN': '국적',
 'SITE_ADDR_RD': '소재지도로명',
 'SITE_ADDR': '소재지지번'}

In [250]:
# 컬럼명 변경하기
df_all.rename(columns=col_dict, inplace=True)
df_all.head(2)

시군구코드 업종코드    년도 업소일련번호      업종명      신고일자     업소명     면적     소재지전화번호  \
강남 0  3220000  211  1983  00001  미용업(일반)  19830627  백민재헤어샵  19.80  02 5451290   
   1  3220000  211  1983  00001  미용업(일반)  19830627  백민재헤어샵  65.67  02 5451290   

        영업자시작일 법인명    소재지시작일 행정동명 폐업일자 폐업사유    업태명  위생교육수료일자  객실수  한실수  양실수  \
강남 0  20110502      20180515  청담동            일반미용업  20110622  0.0  0.0  0.0   
   1  20110502      20180515  청담동            일반미용업  20110622  0.0  0.0  0.0   

      의자수  욕실수 발한실  세탁기수                허가(신고)번호 내외국인구분 국적  \
강남 0  3.0  0.0  N   0.0   3220000-211-1983-00001  내국인        
   1  3.0  0.0  N   0.0   3220000-211-1983-00001  내국인        

                                            소재지도로명  \
강남 0  서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   
   1  서울특별시 강남구 영동대로 702, 화천회관빌딩 지상2층 222-1호 (청담동)   

                                소재지지번  
강남 0  서울특별시 강남구 청담동  133번지 3호  화천회관빌딩  
   1  서울특별시 강남구 청담동  133번지 3호  화천회관빌딩

---
### DF를 csv 파일로 저장

In [251]:
# 오늘 날짜 뽑기
# 파일명 뒤에 붙여줄 건데 기본적으로 '월/일/년' 형태로 뽑힌다.
# '/' 문자는 파일명에 쓸 수 없기 때문에 '-' 문자로 바꿔준다.

date = time.strftime('%x', time.localtime(time.time()))
date = date.replace('/','-')
date

'03-26-20'

In [252]:
# 파일 이름 지정
file_name = './seoul_salon_' + date + ".csv"
file_name

'./seoul_salon_03-26-20.csv'

In [253]:
# csv 파일로 저장하기
df_all.to_csv(file_name, encoding='cp949')